In [23]:
import pandas as pd
import ta

# Substitua 'caminho_do_arquivo.csv' pelo caminho do seu arquivo CSV
caminho_do_arquivo = "C:/Users/Arthur Age/Desktop/USDJPYDaily.csv"

# Lê o arquivo CSV
df = pd.read_csv(caminho_do_arquivo)

# Converte a coluna de data para datetime
df['Date'] = pd.to_datetime(df['Date'])

# Filtra os anos de 2023 a 2025
df_filtered = df[(df['Date'].dt.year >= 2023) & (df['Date'].dt.year <= 2025)]

# Calcula o índice direcional médio de 14 períodos (ADX)
df_filtered['ADX'] = ta.trend.adx(df_filtered['High'], df_filtered['Low'], df_filtered['Close'], window=14)

# Filtra os valores da coluna 'Close' que são maiores que 25 no ADX
result = df_filtered[df_filtered['ADX'] > 25]['Close']

# Exibe o resultado
print(result)

4026    136.229
4027    136.217
4028    136.217
4029    136.752
4030    135.826
         ...   
4555    147.145
4556    147.716
4557    148.185
4558    147.714
4559    148.621
Name: Close, Length: 309, dtype: float64


C:\Users\Arthur Age\AppData\Local\Temp\ipykernel_12008\2874680897.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['ADX'] = ta.trend.adx(df_filtered['High'], df_filtered['Low'], df_filtered['Close'], window=14)


In [24]:
import pandas as pd
import ta

# Substitua 'caminho_do_arquivo.csv' pelo caminho do seu arquivo CSV
caminho_do_arquivo = "C:/Users/Arthur Age/Desktop/USDJPYDaily.csv"

# Lê o arquivo CSV
df = pd.read_csv(caminho_do_arquivo)

# Converte a coluna de data para datetime
df['Date'] = pd.to_datetime(df['Date'])

# Filtra os anos de 2023 a 2025
df_filtered = df[(df['Date'].dt.year >= 2023) & (df['Date'].dt.year <= 2025)]

# Calcula o índice direcional médio de 14 períodos (ADX)
df_filtered['ADX'] = ta.trend.adx(df_filtered['High'], df_filtered['Low'], df_filtered['Close'], window=14)

# Filtra os valores da coluna 'Close' que são maiores que 25 no ADX
filtered_close = df_filtered[df_filtered['ADX'] > 25]['Close']

# Calcula o True Range (TR) de 10 períodos
df_filtered['TR'] = ta.volatility.average_true_range(df_filtered['High'], df_filtered['Low'], df_filtered['Close'], window=10)

# Calcula o stop e o gain
df_filtered['Stop'] = filtered_close - 1.5 * df_filtered['TR']
df_filtered['Gain'] = filtered_close + 4 * df_filtered['TR']

# Exibe o resultado
result = df_filtered[['Date', 'Close', 'Stop', 'Gain']]
print(result)

           Date    Close  Stop  Gain
3986 2023-01-02  130.668   NaN   NaN
3987 2023-01-03  131.011   NaN   NaN
3988 2023-01-04  132.621   NaN   NaN
3989 2023-01-05  133.402   NaN   NaN
3990 2023-01-06  132.138   NaN   NaN
...         ...      ...   ...   ...
4561 2025-03-18  149.230   NaN   NaN
4562 2025-03-19  148.795   NaN   NaN
4563 2025-03-20  148.757   NaN   NaN
4564 2025-03-21  149.298   NaN   NaN
4565 2025-03-24  149.775   NaN   NaN

[580 rows x 4 columns]


C:\Users\Arthur Age\AppData\Local\Temp\ipykernel_12008\2848778879.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['ADX'] = ta.trend.adx(df_filtered['High'], df_filtered['Low'], df_filtered['Close'], window=14)
C:\Users\Arthur Age\AppData\Local\Temp\ipykernel_12008\2848778879.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['TR'] = ta.volatility.average_true_range(df_filtered['High'], df_filtered['Low'], df_filtered['Close'], window=10)
C:\Users\Arthur Age\AppData\Local\Temp\

In [27]:
import pandas as pd
import ta

# Substitua 'caminho_do_arquivo.csv' pelo caminho do seu arquivo CSV
caminho_do_arquivo = "C:/Users/Arthur Age/Desktop/USDJPYDaily.csv"

# Lê o arquivo CSV
df = pd.read_csv(caminho_do_arquivo)

# Converte a coluna de data para datetime
df['Date'] = pd.to_datetime(df['Date'])

# Filtra os anos de 2023 a 2025
df_filtered = df[(df['Date'].dt.year >= 2023) & (df['Date'].dt.year <= 2025)]

# Calcula o índice direcional médio de 14 períodos (ADX)
df_filtered['ADX'] = ta.trend.adx(df_filtered['High'], df_filtered['Low'], df_filtered['Close'], window=14)

# Filtra os valores da coluna 'Close' que são maiores que 25 no ADX
filtered_close = df_filtered[df_filtered['ADX'] > 25]['Close']

# Calcula o True Range (TR) de 10 períodos
df_filtered['TR'] = ta.volatility.average_true_range(df_filtered['High'], df_filtered['Low'], df_filtered['Close'], window=10)

# Calcula o stop e o gain
df_filtered['Stop'] = filtered_close - 1.5 * df_filtered['TR']
df_filtered['Gain'] = filtered_close + 4 * df_filtered['TR']

# Exibe o resultado
result = df_filtered[['Date', 'Close', 'Stop', 'Gain']]

# Salva as primeiras 200 linhas em um arquivo JSON
result.to_json("resultado.json", orient="records", date_format="iso")


C:\Users\Arthur Age\AppData\Local\Temp\ipykernel_12008\466490865.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['ADX'] = ta.trend.adx(df_filtered['High'], df_filtered['Low'], df_filtered['Close'], window=14)
C:\Users\Arthur Age\AppData\Local\Temp\ipykernel_12008\466490865.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['TR'] = ta.volatility.average_true_range(df_filtered['High'], df_filtered['Low'], df_filtered['Close'], window=10)
C:\Users\Arthur Age\AppData\Local\Temp\ip